In [1]:
import numpy as np
import pandas as pd

### Parámetros de entrada
Información que obtiene el modelo del sistema de información

In [2]:
# Demanda
DEMANDA = 60
# Costo por unidad de tiempo
CTIEMPO = 100
# Tiempo máximo definido
T_MAX = 360

# Parámetros de entrada
params_df = pd.read_excel('./data/' + 'nuevo_info_acopios.xlsx')

# Matriz de costos de transporte
costo_df = pd.read_excel('./data/' + 'costoTransporteCAi.xlsx', index_col=0)

# Matriz de tiempo de transporte
tiempo_df = pd.read_excel('./data/' + 'tiempoTransporteCAi.xlsx', index_col=0)

### Variables
Definición de variables para el modelo de optimización

In [3]:
N = params_df.shape[0]
capacidades = np.empty(N * 2, dtype=float)

for cap_i in range(0, N * 2, 2):
    capacidades[cap_i] = params_df['Stock'].iloc[cap_i // 2]
    capacidades[cap_i + 1] = params_df['Ppotencial'].iloc[cap_i // 2]
    
capacidades

array([10.   ,  0.5  , 12.   ,  0.625,  4.   , 14.   ,  8.   , 13.   ,
       18.   , 15.   ,  6.   , 14.   , 12.   , 13.   ,  2.   , 12.5  ,
       30.   , 15.   , 15.   , 18.   ])

# Función objetivo
$$
\begin{align*}
    \sum_{i=1 \quad i\neq p}^{N} &\big[ \, kCA_i \times Precio(CA_i) + kCA_i \times cTransp(CA_i) + Tiempo(CA_i) \times cTiempo \, \big] \, + \\
                                 &\big[ \, kCA_p \times Precio(CA_p) + Demanda \times cTransp(CA_p) + Tiempo(CA_p) \times cTiempo \, \big]
\end{align*}
$$

In [4]:
def f(x, idx_principal=0):
    delta = 0
    
    for i in range(0, N*2, 2):
        idx_acopio = i // 2

        if i == idx_principal or (x[i] == 0 and x[i+1] == 0):
            continue
            
        delta += get_delta(x, idx_acopio, idx_principal)

    delta += get_delta(x, idx_principal)
    
    return delta

def get_delta(x, idx_acopio, idx_principal=-1):
    kca = x[idx_acopio] + x[idx_acopio + 1]
    precio = params_df['Precio'].iloc[idx_acopio]
    talistam = params_df['TiempoAlistam'].iloc[idx_acopio]

    # Si es el centro de acopio principal (idx=-1)
    if idx_principal < 0:
        ctransp = params_df['Ctransp'].iloc[idx_acopio]
        ttransp = params_df['TiempoTransp'].iloc[idx_acopio]
        tiempo = talistam + ttransp
        
        return (kca * precio) + (DEMANDA * ctransp) + (tiempo * CTIEMPO)
    else:
        ctransp = costo_df.iloc[idx_acopio, idx_principal]
        ttransp = tiempo_df.iloc[idx_acopio, idx_principal]
        tiempo = talistam + ttransp
        
        return (kca * precio) + (kca * ctransp) + (tiempo * CTIEMPO)


# f((0, 0, 0, 0, 4, 0, 8, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))

Sujeta a las restricciones:
$$
\begin{align*}
    \sum_{i=0}^{N} kCA_i &= Demanda \\
    kCA_i &\leq Stock(CA_i) + Ppotencial(CA_i) &\therefore \, i=0,\cdots ,N \\
    TiempoAlistam(CA_i) &\leq TiempoMaxDefinido &\therefore \, i=0,\cdots ,N \\
    Tiempo(CA_i) &= TiempoAlistam(CA_i) + TiempoTransp(CA_i) \\
\end{align*}
$$

## scipy

In [5]:
# c = tuple(idx for (_, idx) in bounds)
# Aeq = np.ones((1, len(c)))
# beq = 1

In [6]:
# from scipy.optimize import linprog
# # from scipy.optimize import differential_evolution
# 
# result = linprog(
#     c,
#     A_ub=Aeq,
#     b_ub=beq,
#     bounds=bounds,
#     method='highs',
#     integrality=1
# )
# print(result)

## pymoo

Iterar el modelo n veces según centros de acopio haya. La idea es asumir en cada iteración que el cacopio principal sea cada uno, desde i=0 hasta n.

Se evalúa el modelo tomando cada centro de acopio como el principal, y la ejecución da resultados para cada uno, luego de todas las soluciones se toma la mejor solución.

**Costo a optimizar: 1.981.725** 

In [7]:
from pymoo.core.problem import ElementwiseProblem

xl = np.zeros(N*2)
xu = capacidades

class Queso(ElementwiseProblem):
    def __init__(self):
        super().__init__(
            n_var=len(xl),
            n_obj=1,
            n_eq_constr=1,
            # n_ieq_constr=1,
            xl=xl,
            xu=xu
        )
        
    def _evaluate(self, x, out, *args, **kwargs):
        out['F'] = f(x)
        out['H'] = DEMANDA - np.sum(x)

In [8]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.repair.rounding import RoundingRepair

model = Queso()

algorithm = GA(
    # pop_size=200,
    sampling=IntegerRandomSampling(),
    crossover=SBX(prob=1.0, eta=3.0, vtype=float, repair=RoundingRepair()),
    mutation=PM(prob=1.0, eta=3.0, vtype=float, repair=RoundingRepair()),
    eliminate_duplicates=True
)

res = minimize(model,
               algorithm,
               # termination,
               seed=1,
               verbose=False)

print(f'F: {res.F}, \nX: {res.X}')


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False

F: [2029383.], 
X: [ 1  0  2  0  0  0  2  3  0  2  1 12  8  3  0  6  5  0 14  1]


In [9]:
# Alterar elementos en un individuo, esto puede usarse para mutación en GA y SA

# Valores en cero
def alter_lower(single, idx_mut, capacidades):
    delta = capacidades[idx_mut]
    ca = list(np.nonzero(single)[0])
    idx = -1
    
    single[idx_mut] = delta
    
    while np.sum(single) != DEMANDA:
        if len(ca) > 0:
            idx = random.choice(ca)
            ca.remove(idx)
        
        if single[idx] - delta >= 0:
            single[idx] -= delta
        else:
            delta -= single[idx]
            single[idx] = 0
            

# Valores no cero
def alter_upper(single, idx_mut, capacidades):
    delta = single[idx_mut]
    ca = list(np.where(single == 0)[0])
    idx = -1
    
    single[idx_mut] = 0
    
    while np.sum(single) < DEMANDA:
        if len(ca) > 0:
            idx = random.choice(ca)
            ca.remove(idx)
        
        if capacidades[idx] == single[idx]:
            continue
        
        if single[idx] + delta > capacidades[idx]:
            delta -= capacidades[idx] - single[idx]
            single[idx] = capacidades[idx]
        else:
            single[idx] += delta
            delta = 0

## Generación 
De nuevas posibles soluciones

In [10]:
from pymoo.core.sampling import Sampling
import random

class TopOrZeroSampling(Sampling):
    def _do(self, problem, n_samples, **kwargs):
        vec = np.zeros(problem.n_var, dtype=float)
        
        while np.sum(vec) < DEMANDA:
            idx = np.random.randint(problem.n_var)
            vec[idx] += random.choice((0, capacidades[idx]))
            
            if np.sum(vec) > DEMANDA:
                vec[idx] = vec[idx] - (np.sum(vec) - DEMANDA)
                
        return [vec,]

## Cruce
1. Tomar dos individuos aleatorios como padres
2. Utilizar un índice separador (la mitad)
3. Crear un nuevo individuo, para 

In [11]:
from pymoo.core.crossover import Crossover


        

## Mutación
1. Tomar un índice (cacopio) aleatorio
2. **Si es cero:**
3. Cambiar a la cantidad máxima de la capacidad del centro de acopio
4. La cantidad que se asignó, restarla a lo largo de los demás centros de acopio para mantener la restricción de igualdad
5. **Sino:**
6. Cambiar a cero
7. La cantidad que se eliminó, sumarla en los demás centros de acopio para mantener la restricción de igualdad

In [12]:
from pymoo.core.mutation import Mutation

class Mut(Mutation):
    def __init__(self):
        super().__init__()

    def _do(self, problem, X, **kwargs):
        for i in range(len(X)):
            r = np.random.random()

            if r < 0.4:
                idx_mut = np.random.randint(X.shape[1])

                if X[i][idx_mut] == 0:
                    alter_lower(X[i], idx_mut, capacidades)

                else:
                    alter_upper(X[i], idx_mut, capacidades)
        return X


In [27]:
class Cross(Crossover):
    def __init__(self):
        # Parents, Offspring
        super().__init__(2, 1)

    def _do(self, problem, X, **kwargs):
        # n_parents, n_matings, n_var
        _, n_matings, n_var = X.shape
        
        Y = np.zeros((1, n_matings, n_var))
        
        for idx in range(n_matings):
            p1 = X[0, idx, : n_var//2]
            p2 = X[1, idx, n_var//2 : n_var]
            child = np.concatenate((p1, p2))
            Y[0, idx, :] = child
            
        return Y

algorithm = GA(
    pop_size=3,
    sampling=TopOrZeroSampling(),
    crossover=Cross(),
    # mutation=Mut(),
    eliminate_duplicates=True
)

res = minimize(model,
               algorithm,
               seed=1,
               verbose=True)

print(f'F: {res.F}, \nX: {res.X}')

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |        1 |  0.000000E+00 |  0.000000E+00 |  3.157650E+06 |  3.157650E+06
     2 |        4 |  0.000000E+00 |  0.3305861576 |  3.157650E+06 |  3.157650E+06
     3 |        7 |  0.000000E+00 |  0.2052635391 |  3.157650E+06 |  3.157650E+06
     4 |       10 |  0.000000E+00 |  0.0936174631 |  3.157650E+06 |  3.157650E+06
     5 |       13 |  0.000000E+00 |  0.0299106396 |  3.157650E+06 |  3.157650E+06
     6 |       16 |  0.000000E+00 |  0.0299106396 |  3.157650E+06 |  3.157650E+06
     7 |       19 |  0.000000E+00 |  0.0176448643 |  3.157650E+06 |  3.157650E+06
     8 |       22 |  0.000000E+00 |  0.0176448643 |  3.157650E+06 |  3.157650E+06
     9 |       25 |  0.000000E+00 |  0.0176448643 |  3.157650E+06 |  3.157650E+06
    10 |       28 |  0.000000E+00 |  0.0176448643 |  3.157650E+06 |  3.157650E+06
    11 |       31 |  0.000000E+00 |  0.0093242757 |  3.157650E+06 |  3.157650E+06
    12 |       3